# LangChain Academy

Welcome to LangChain Academy!

## Context

At LangChain, we aim to make it easy to build LLM applications. One type of LLM application you can build is an agent. There’s a lot of excitement around building agents because they can automate a wide range of tasks that were previously impossible.

In practice though, it is incredibly difficult to build systems that reliably execute on these tasks. As we’ve worked with our users to put agents into production, we’ve learned that more control is often necessary. You might need an agent to always call a specific tool first or use different prompts based on its state.

To tackle this problem, we’ve built [LangGraph](https://langchain-ai.github.io/langgraph/) — a framework for building agent and multi-agent applications. Separate from the LangChain package, LangGraph’s core design philosophy is to help developers add better precision and control into agent workflows, suitable for the complexity of real-world systems.

## Course Structure

The course is structured as a set of modules, with each module focused on a particular theme related to LangGraph. You will see a folder for each module, which contains a series of notebooks. A video will accompany each notebook to help walk through the concepts, but the notebooks are also stand-alone, meaning that they contain explanations and can be viewed independent of the videos. Each module folder also contains a `studio` folder, which contains a set of graphs that can be loaded into [LangGraph Studio](https://github.com/langchain-ai/langgraph-studio), our IDE for building LangGraph applications.

## Setup

Before you begin, please follow the instructions in the `README` to create an environment and install dependencies.

## Chat models

In this course, we'll be using [Chat Models](https://python.langchain.com/v0.2/docs/concepts/#chat-models), which do a few things take a sequence of messages as inputs and return chat messages as outputs. LangChain does not host any Chat Models, rather we rely on third party integrations. [Here](https://python.langchain.com/v0.2/docs/integrations/chat/) is a list of 3rd party chat model integrations within LangChain! By default, the course with use [GEMINI_API_KEY](https://ai.google.dev/gemini-api/docs/api-key/) because it is both popular and performant. As noted, please ensure that you have an `GEMINI_API_KEY`.

Let's check that your `GEMINI_API_KEY` is set and, if not, you will be asked to enter it.

In [1]:
%%capture --no-stderr
!pip install -q langchain_google_genai langchain_core langchain_community tavily-python

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key=GEMINI_API_KEY,
    temperature=0
)


In [7]:
result  = llm.invoke("Who won 2024 US presidential ellection?")
result

AIMessage(content='The 2024 US presidential election has not yet happened.  The election will be held in November 2024.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-2eeeb40a-d8ce-4a0d-b6ff-10feae538e89-0', usage_metadata={'input_tokens': 13, 'output_tokens': 29, 'total_tokens': 42, 'input_token_details': {'cache_read': 0}})

Chat models in LangChain have a number of [default methods](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface). For the most part, we'll be using:

* `stream`: stream back chunks of the response
* `invoke`: call the chain on an input

And, as mentioned, chat models take [messages](https://python.langchain.com/v0.2/docs/concepts/#messages) as input. Messages have a role (that describes who is saying the message) and a content property. We'll be talking a lot more about this later, but here let's just show the basics.

In [8]:
from langchain_core.messages import HumanMessage, AIMessage

# Create a message
# Message list
messages = [
    HumanMessage(content="Hi", name="Human Student"),
    AIMessage(content='Hi! How can I help you today? \n', name="AI Assistant"),
    HumanMessage(content="What is LangChain?", name="Human Student"),
    AIMessage(content='LangChain is a framework for developing applications powered by language models.', name="AI Assistant"),
    HumanMessage(content="How can I learn", name="Human Student"),
    ]

# Invoke the model with a list of messages
llm.invoke(messages)

AIMessage(content='There are several ways to learn LangChain, depending on your learning style and prior experience:\n\n**1. Official Documentation:** The best place to start is the official LangChain documentation.  It\'s well-structured and provides comprehensive tutorials and examples.  Look for their "Getting Started" guides and work your way through the examples.\n\n**2. Tutorials and Blog Posts:** Many tutorials and blog posts are available online that cover various aspects of LangChain. Search for "LangChain tutorial" or "LangChain example" on Google, YouTube, and Medium.  Look for tutorials that focus on specific use cases that interest you, such as building a chatbot or a question-answering system.\n\n**3. GitHub Repository:** Explore the LangChain GitHub repository.  You can find the source code, contribute to the project, and see how different components are implemented.  Reading the code can be a great way to deepen your understanding.\n\n**4. Example Projects:**  The LangC

We get an `AIMessage` response. Also, note that we can just invoke a chat model with a string. When a string is passed in as input, it is converted to a `HumanMessage` and then passed to the underlying model.


The interface is consistent across all chat models and models are typically initialized once at the start up each notebooks.

So, you can easily switch between models without changing the downstream code if you have strong preference for another provider.


## Search Tools

You'll also see [Tavily](https://tavily.com/) in the README, which is a search engine optimized for LLMs and RAG, aimed at efficient, quick, and persistent search results. As mentioned, it's easy to sign up and offers a generous free tier. Some lessons (in Module 4) will use Tavily by default but, of course, other search tools can be used if you want to modify the code for yourself.

In [9]:
import os
from google.colab import userdata
TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

from langchain_community.tools.tavily_search import TavilySearchResults
tavily_search = TavilySearchResults(max_results=3)
search_docs = tavily_search.invoke("Who won 2024 US presidential ellection?")

SecretNotFoundError: Secret TAVILY_API_KEY does not exist.

In [ ]:
search_docs

[{'url': 'https://www.bbc.com/news/election/2024/us/results',
  'content': 'US Presidential Election Results 2024 - BBC News Close menu BBC News Kamala Harris of the Democrat party has 0 electoral college votes. Donald Trump of the Republican party has 0 electoral college votes. Kamala Harris of the Democrat party has 158,810 votes (38.4%) Donald Trump of the Republican party has 249,225 votes (60.2%) US presidential election results 2024 US election 2024 Voting in some states is particularly hard to predict, with polls showing they could be won by the Republicans or the Democratic party. The battleground states that could decide the 2024 presidential election are: Voters in 11 states will also elect a governor. US election polls: Who is ahead - Harris or Trump? US election 2024 About the BBC'},
 {'url': 'https://www.270towin.com/',
  'content': 'Headlines\nGeorge Santos Expelled from House of Representatives\n311 members voted to remove the New York Republican from office\nCeleste Mal